In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, scf, fci

import sys
util_dir = "/home/jwdesroches/python/Ga2QuAMES/symmetry/helper_funcs/"
sys.path.append(util_dir)
from utils import *
plt.rcParams.update({'font.size': 16})

In [60]:
df = pd.read_csv("extracted_data.csv")
df = df.dropna()

df.head()

,Filename,Molecule,Basis Set,Geometry,Charge,Spin,R value,Symmetry Case,Bond Length,Number of Qubits [JW Hamiltonian],Number of Qubits [Ising Hamiltonian],Ising Energy [Hartree],Total Calculation Time [s]
0,input-0001.out,H2,sto-3g,H 0 0 0; H 0 0 0.3,0.0,0.0,1.0,yes,0.3,1.0,1.0,-0.593828,0.001604
1,input-0002.out,H2,sto-3g,H 0 0 0; H 0 0 0.4,0.0,0.0,1.0,yes,0.4,1.0,1.0,-0.904361,0.001886
2,input-0003.out,H2,sto-3g,H 0 0 0; H 0 0 0.5,0.0,0.0,1.0,yes,0.5,1.0,1.0,-1.042996,0.001574
3,input-0004.out,H2,sto-3g,H 0 0 0; H 0 0 0.6,0.0,0.0,1.0,yes,0.6,1.0,1.0,-1.101128,0.002512
4,input-0005.out,H2,sto-3g,H 0 0 0; H 0 0 0.7,0.0,0.0,1.0,yes,0.7,1.0,1.0,-1.117349,0.002122


In [67]:
def plot_results(mol_name, basis_set, include_FCI=True, include_FCI_comparison=True, include_high_r = False):

    if include_high_r:
        if mol_name == "H2":
            possible_rs = [1,2,3,4]
        elif mol_name == "He2":
            possible_rs = [4,8,12,16]
        else: 
            print("High r values are not available for that molecule.")
            possible_rs = return_r_values(mol_name, "yes")
    else:
        possible_rs = return_r_values(mol_name, "yes")

    if include_FCI:
        fci_bond_lengths = np.linspace(0.3, 5.0, 100)
        fci_energies = []
        for bond_length in fci_bond_lengths:
            mol = gto.Mole()
            mol.build(atom=return_geometry(mol_name, bond_length), basis=basis_set, unit="angstrom")
            mf = scf.RHF(mol)
            mf.kernel()

            cisolver = fci.FCI(mf)
            fci_energy = cisolver.kernel()[0]
            fci_energies.append(fci_energy)

    if include_FCI_comparison:
        fci_bond_lengths_comp = np.around(np.arange(0.3, 5.0, 0.1), 2)
        sym_diffs_dict = {r: [] for r in possible_rs}
        nosym_diffs_dict = {r: [] for r in possible_rs}
        for bond_length in fci_bond_lengths_comp:
            try:
                mol = gto.Mole()
                mol.build(atom=return_geometry(mol_name, bond_length), basis=basis_set, unit="angstrom")
                mf = scf.RHF(mol)
                mf.kernel()

                cisolver = fci.FCI(mf)
                fci_energy = cisolver.kernel()[0]
            except Exception as e:
                fci_energy = np.nan
            
            for r in possible_rs:
                sym_diff = df["Ising Energy [Hartree]"][(df["Molecule"] == mol_name) & 
                                                        (df["Basis Set"] == basis_set) & 
                                                        (df["Symmetry Case"] == "yes") & 
                                                        (df["R value"] == r) & 
                                                        (df["Bond Length"] == bond_length)].values
                nosym_diff = df["Ising Energy [Hartree]"][(df["Molecule"] == mol_name) & 
                                                        (df["Basis Set"] == basis_set) & 
                                                        (df["Symmetry Case"] == "no") & 
                                                        (df["R value"] == r) & 
                                                        (df["Bond Length"] == bond_length)].values
                
                sym_diff = sym_diff[0] - fci_energy if sym_diff.size > 0 else np.nan
                nosym_diff = nosym_diff[0] - fci_energy if nosym_diff.size > 0 else np.nan
                
                sym_diffs_dict[r].append(sym_diff)
                nosym_diffs_dict[r].append(nosym_diff)

    fig, axs = plt.subplots(nrows=1, ncols=len(possible_rs), figsize=(7 * len(possible_rs), 5), sharex=True, sharey=True)
    if len(possible_rs) == 1:
        ax = axs
    else:
        axs = axs.flatten()
    if len(possible_rs) == 1:
        axs = [axs]
    
    if include_FCI_comparison:
        fig_diffs, axs_diffs = plt.subplots(nrows=1, ncols=len(possible_rs), figsize=(7 * len(possible_rs), 3), sharex=True, sharey=True)
        if len(possible_rs) == 1:
            axs_diffs = [axs_diffs]
        else:
            axs_diffs = axs_diffs.flatten()
    
    for i, r in enumerate(possible_rs):
        ax = axs[i]
        ax.plot(df["Bond Length"][(df["Molecule"] == mol_name) & (df["Basis Set"] == basis_set) & 
                    (df["Symmetry Case"] == "yes") & (df["R value"] == r)], 
                df["Ising Energy [Hartree]"][(df["Molecule"] == mol_name) & (df["Basis Set"] == basis_set) & 
                    (df["Symmetry Case"] == "yes") & (df["R value"] == r)], ".r", markersize = 10, label="w/ SAE")

        if len(df["Ising Energy [Hartree]"][(df["Molecule"] == mol_name) & (df["Basis Set"] == basis_set) & 
                    (df["Symmetry Case"] == "no") & (df["R value"] == r)]) != 0:
        
            ax.plot(df["Bond Length"][(df["Molecule"] == mol_name) & (df["Basis Set"] == basis_set) & 
                        (df["Symmetry Case"] == "no") & (df["R value"] == r)], 
                    df["Ising Energy [Hartree]"][(df["Molecule"] == mol_name) & (df["Basis Set"] == basis_set) & 
                        (df["Symmetry Case"] == "no") & (df["R value"] == r)], "xb", markersize = 8, label="w/o SAE")
    
        if mol_name == "H2":
        
            ax.text(0.5, 0.125, f"$r=${r}", transform=ax.transAxes, fontsize=20, verticalalignment="top", 
                    bbox=dict(boxstyle='square,pad=0.3', edgecolor='black', facecolor='white'))
            
        else:
            ax.text(0.5, 0.95, f"$r=${r}", transform=ax.transAxes, fontsize=20, verticalalignment="top", 
                    bbox=dict(boxstyle='square,pad=0.3', edgecolor='black', facecolor='white'))
        
        if include_FCI:
            ax.plot(fci_bond_lengths, fci_energies, '.k', label="FCI")
            ax.plot(fci_bond_lengths, fci_energies, '-k')
        
        #ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
                  #ncol=3, fancybox=True, shadow=True)
        
        ax.set_xlabel("Bond Length [$\AA$]")
        ax.set_ylabel("Energy [$E_h$]")
        ax.grid()
        ax.set_xlim(0.2,5.1)
        #ax.set_ylim(-56.5, -54)
        
        if include_FCI_comparison:
            sym_diffs = sym_diffs_dict[r]
            nosym_diffs = nosym_diffs_dict[r]
            axs_diffs[i].plot(fci_bond_lengths_comp, np.abs(sym_diffs), '.r', markersize = 10, label="w/ SAE")
            axs_diffs[i].plot(fci_bond_lengths_comp, np.abs(nosym_diffs), 'xb', markersize = 8, label="w/o SAE")
            axs_diffs[i].plot([],[],".k", label = "FCI")
            
            axs_diffs[i].set_xlabel("Bond Length [$\AA$]")
            axs_diffs[i].set_ylabel("$\Delta E$ [$E_h$]")
            axs_diffs[i].grid()
            axs_diffs[i].set_xlim(0.2, 5.1)
            
        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5125, 1.12), ncol=3, fancybox=True, shadow=True)

    fig.tight_layout()
    if include_FCI_comparison:
        fig_diffs.tight_layout()
    plt.show()

    return None

In [68]:
mol_name = "H2"
basis_set = "sto-6g"

"""mol_names = "He2, "
for mol_name in mol_names:
    """
    
    

# need to remake figures for He2, LiH, H2O, BH3, NH3, CH4, F2, Li2, N2, O2, CO

plot_results(mol_name, basis_set, include_FCI = True, include_FCI_comparison = True, include_high_r=False)

IndentationError: expected an indented block after 'for' statement on line 5 (2393589365.py, line 12)